In [2]:
from langchain_google_genai import GoogleGenerativeAI
import pandas as pd
llm=GoogleGenerativeAI(model="gemini-1.5-flash")

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
name=pd.ExcelFile(r"D:\New Folder\multi_sheet_excel.xlsx").sheet_names
name

['Orders', 'Customers', 'Sales Report']

In [111]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
def sheet_finder(question,sheet_name):
    temp = """Based on the given question: {question}
If user question is inappropriate say {{"response":"Provide more details"}}
Determine the most relevant sheet name(s) that best suit the user's query.

- If the question **requires comparison** between multiple categories, provide **multiple sheet names** that contain the necessary data.
- If the question **requires data merging or joining** (e.g., relationships between entities across sheets), identify the sheets that should be combined.
- If the question refers to a **single topic**, provide **only one** sheet name.
- If no exact match is found, suggest the closest possible sheet(s).
- If we reuqire two tables then comparison must be true.
Analyze this JSON Data: {name}

Just provide the sheet name(s) as a list. If a join is needed, indicate which sheets should be merged:
Output Format:
{{
    sheet_name:[],
    Join on:{{}},
    comparison: True or false
}}
"""
    prompt=PromptTemplate(input_variables=["question","name"],template=temp)
    chain= LLMChain(prompt=prompt,llm=llm)
    response = chain.invoke({"question":question,"name":sheet_name})['text']
    response = response.replace("```json","").replace("```","")
    return response
    



In [112]:
column_name={}
for i in name:
    df=pd.read_excel(r"D:\New Folder\multi_sheet_excel.xlsx",sheet_name=i)
    column_name[i]=df.columns
column_name

{'Orders': Index(['Order ID', 'Customer Name', 'Product', 'Quantity', 'Price',
        'Order Date'],
       dtype='object'),
 'Customers': Index(['Customer ID', 'Name', 'Email', 'Phone', 'Address'], dtype='object'),
 'Sales Report': Index(['Month', 'Total Sales', 'Profit'], dtype='object')}

In [113]:
import json
question="What is the total number of unique products ordered?"
js=sheet_finder(question,column_name)
js=json.loads(js)
print(js)

{'sheet_name': ['Orders'], 'Join on': {}, 'comparison': False}


In [114]:
def merge_excel(file_path,merge_info,how="outer"):
    sheet_names=merge_info['sheet_name']
    join_key=merge_info['Join on']
    dfs={sheet:pd.read_excel(file_path,sheet_name=sheet) for sheet in sheet_names}
    merged_df = dfs[sheet_names[0]]

    # Dynamically merge all sheets based on their respective join keys
    for i in range(len(sheet_names) - 1):
        left_key, right_key = join_key[sheet_names[i]], join_key[sheet_names[i + 1]]
        merged_df = merged_df.merge(dfs[sheet_names[i + 1]], left_on=left_key, right_on=right_key, how=how)

    return merged_df
    

In [115]:
df=merge_excel(r"D:\New Folder\multi_sheet_excel.xlsx",js)

In [116]:
df

,Order ID,Customer Name,Product,Quantity,Price,Order Date
0,101,Alice,Laptop,1,1200,2024-01-10
1,102,Bob,Phone,2,800,2024-01-12
2,103,Charlie,Tablet,1,500,2024-01-15
3,104,David,Headphones,3,150,2024-01-18


In [117]:
def excel_table(question,df):
    list1=[]
    for i in df.columns:
        list1.append(df[i].dtype)
    prompt = """You are an expert in Python and Pandas. Generate Python code based on the user's question: {question}, using the given DataFrame `external_df`.

### Guidelines:
1. **DataFrame Details**:
   - Column names: {df}
   - Column datatypes: {datatype}
   - Sample data: {sample}
   - Null values for column: {null_values} 
   - fill null values with empty string.
2. The output must be a Pandas DataFrame named `output_df`. If no relevant columns match the question, set `output_df = None`.
3. Use explicit imports for all necessary libraries (e.g., `import pandas as pd`) to prevent errors.
4. Do not create a sample data or assume sample data, alter the DataFrame, or include any functionality beyond answering the question.
5. Include concise comments in your code for clarity.

### Key Points:
- Use .lower for matching the strings becuase their might be chance that the data should consist of some upper case or captilaize type of strings.
- Output only the Python code; no explanations, text, or examples.
- If the question is unrelated to the DataFrame, return `None` for `output_df`.
- No Need to write functions
# Your code starts below:
    """
    prompt = PromptTemplate(template=prompt
    , input_variables=["question","df","datatype","sample","null_values"])
    prompt_formatted_str = prompt.format(
        question=question,df=df.columns,datatype=list1,sample=df.sample(),null_values=df.isnull().sum()
    )
    prediction = llm.invoke(prompt_formatted_str)
    # prediction =prediction.text
    prediction=prediction.replace("```python","")
    prediction=prediction.replace("```","")
    return prediction


In [118]:
question="Find the total revenue generated by each customer."
js=sheet_finder(question,column_name)
js=json.loads(js)
print(js)

{'sheet_name': ['Orders', 'Customers'], 'Join on': {'Orders': 'Customer Name', 'Customers': 'Name'}, 'comparison': False}


In [119]:
if len(js['sheet_name'])< 2:
    df=pd.read_excel(r"D:\New Folder\multi_sheet_excel.xlsx",sheet_name=js['sheet_name'][0])
    view=excel_table(question,df)
elif js['Join on'] != {}:
    df=merge_excel(r"D:\New Folder\multi_sheet_excel.xlsx",js)
    view=excel_table(question,df)
elif js['comparison'] == True or len(js['sheet_name']>2):
    length=len(js['sheet_name'])
    print(length)

In [120]:
external_df=df
local_vars={"external_df":df,"output_df":None}
exec(view,local_vars)

In [121]:
local_vars['output_df']

,Customer Name,Total Revenue
0,Alice,1200
1,Bob,1600
2,Charlie,500
3,David,450


In [122]:
question="What if"
js=sheet_finder(question,column_name)
js
# js=json.loads(js)


'Here\'s a function to analyze the user\'s question and return the relevant sheet names in the specified format.  It handles various scenarios, including comparisons, joins, and single-topic queries.  Because the JSON data only provides column names, the function relies on keyword matching to determine sheet relevance.  A more sophisticated approach would require semantic understanding of the question.\n\n\npython\nimport json\n\ndef analyze_query(question, data):\n    """\n    Analyzes a user\'s question and determines the relevant sheet names.\n\n    Args:\n        question: The user\'s question (string).\n        data: A dictionary containing sheet names and their column names.\n\n    Returns:\n        A dictionary in the specified output format.  Returns an error message if the question is inappropriate.\n    """\n\n    question = question.lower()  # case-insensitive matching\n\n    if "inappropriate" in question or "offensive" in question:  #Example of inappropriate check - needs 

In [123]:
if len(js['sheet_name'])<2:
    df=pd.read_excel(r"D:\New Folder\multi_sheet_excel.xlsx",sheet_name=js['sheet_name'][0])
    view=excel_table(question,df)
elif js['Join on'] != {}:
    df=merge_excel(r"D:\New Folder\multi_sheet_excel.xlsx",js)
    view=excel_table(question,df)
elif js['comparison'] == True or len(js['sheet_name'])>1:
    length=len(js['sheet_name'])
    views={}
    for i in range(length):
        df=pd.read_excel(r"D:\New Folder\multi_sheet_excel.xlsx",sheet_name=js['sheet_name'][i])
        local_vars={"external_df":df,"output_df":None}
        cod=excel_table(question,df)
        print(cod)
        exec(cod,local_vars)
        views[i]=local_vars['output_df']
else:
    print("Please Enter a Valid question OR Add More depth in your question")


TypeError: string indices must be integers, not 'str'

In [ ]:
views

{0:    Year  Month     Product  Products Sold  Total Sales
 0  2024      1  Headphones              3         2650
 1  2024      1      Laptop              1         2650
 2  2024      1       Phone              2         2650
 3  2024      1      Tablet              1         2650,
 1:       month  total_sales_sum  number_of_products_sold
 0     April             8000                        1
 1  February             7000                        1
 2   January             5000                        1
 3     March             6500                        1}